In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app.database import SessionLocal, engine, Base
import sqlite3
import datetime
from selenium.common.exceptions import NoSuchElementException

In [3]:
from web_scrapping_helpers import FIEPage, parse_metadata, parse_pool, pool_to_db, tournament_to_db, parse_name, round_to_db

In [4]:
def init_db():
    connection = sqlite3.connect('fencing_test.db')
    with open('schema.sql') as f:
        connection.executescript(f.read())
    connection.close()

In [5]:
def get_db_connection():
    conn = sqlite3.connect('fencing_test.db')
    conn.row_factory = sqlite3.Row
    return conn

In [6]:
def parse_tournament(metadata, Scraper):
    conn = get_db_connection()
    comp_meta, link = metadata
    tournament_id = tournament_to_db(comp_meta, conn)
    pool_data = Scraper.to_results().to_pools().get_pools_data()
    if len(pool_data) == 0:
        return 1
    pool_names = Scraper.get_pool_names()
    offset = 0
    for pool in pool_data:
        bouts, pool_size = parse_pool(pool, pool_names[offset:], conn)
        offset += pool_size
        pool_to_db(bouts, tournament_id, conn)

    conn.commit()
    Scraper.to_prelims()
    prelim_data = set(Scraper.get_prelim_data())
    try:
        Scraper.next_round_prelim()
    except NoSuchElementException:
        Scraper.next_round_prelim(option=True)
    finally:
        prelim_data2 = set(Scraper.get_prelim_data())
        prelim_bouts = prelim_data.union(prelim_data2)

        if prelim_data != prelim_bouts:
            rounds = [256, 128, 64]
        else:
            rounds = [128, 64]
        prelim_bouts = list(prelim_bouts)

        pos = 0
        for rnd in rounds:
            num_bouts = rnd//2
            round_to_db(prelim_bouts[pos:pos+num_bouts], f"P{rnd}", tournament_id, conn)
        """Prelim Parsing Logic"""
        Scraper.to_tableu()
        tableu_data = Scraper.get_de_data()
        for i in range(2):
            Scraper.next_round_de().next_round_de()
            next_round_data = Scraper.get_de_data()
            tableu_data.extend(next_round_data)
        rounds = [64, 32, 16, 8, 4, 2]
        pos = 0
        for rnd in rounds:
            num_bouts = rnd//2
            round_to_db(tableu_data[pos:pos+num_bouts], str(rnd), tournament_id, conn)
            #print(f"Round {rnd}:\n{tableu_data[pos:pos+num_bouts]}")
            pos += num_bouts


        conn.commit() 
        conn.close()
        return 0



In [7]:
def scrape_cat(cat):
    Scraper = FIEPage()
    Scraper.open_competions().previous().age_group('s').comp_category(cat).select_weapon('epee').select_gender('m').select_event_type().search()
    # session = SessionLocal
    # session.begin()
    tries = 0
    while tries < 3:
        metadata = Scraper.get_competitions()

    
        for md in metadata:
            if tries < 3:
                m, link = md
                Scraper.open_window(link)
                main, new = Scraper.driver.window_handles
                Scraper.driver.switch_to.window(new)
                handles = Scraper.driver.window_handles
                tries += parse_tournament(md, Scraper)
                Scraper.driver.close()
                Scraper.driver.switch_to.window(main)
        Scraper.next_page_tournaments()
        
    

In [8]:
#init_db()

In [10]:
scrape_cat("A")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\liamc\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


['MINOBE', 'Kazuyasu']
['PAOLINI', 'Giacomo']
['HADZIC', 'Alen']
['BIELEC', 'Maciej']
['ELSOKKARY', 'Ahmad']
['GALUASHVILI', 'Lasha']
['NURMATOV', 'Javokhirbek']
['CIMINI', 'Gabriele']
['KHVEDELIDZE', 'Vladimeri']
['SOCHA', 'Jan']
['ITO', 'Inochi']
['SVENSSON', 'Jonathan']
['OSWALD', 'Charles-Eric']
['VALLOSIO', 'Andrea']
['MCDOWALD', 'Curtis']
['FABREGAT', 'Angel']
['TYCHLER', 'Pavel']
['OLSEN', 'Johannes', 'Kristian']
['GROVER', 'Elliott']
['CHKHETIANI', 'Tornike']
['YETER', 'Yalgin']
['SANER', 'Harry']
['DU', 'Jiaming']
['GORBACHUK', 'Yevgeniy']
['FAVA', 'Alex']
['HADDAD', 'Justin']
['KONGSTAD', 'Conrad', 'Seibaek']
['ESENALIEV', 'Zhoodat']
['METRAILLER', 'Clement']
['ANTKIEWICZ', 'Mateusz']
['BRINKMANN', 'Marco']
['KNUDSEN', 'Kenneth']
['FREILICH', 'Yuval', 'Shalom']
['KHIJAKADZE', 'Gaga']
['PAVLOV', 'Kiril']
['DI', 'VEROLI', 'Davide']
['DIDIA', 'Giorgi']
['VON', 'DER', 'OSTEN', 'Frederik']
['MOHAMED', 'Mayed']
['NIGGELER', 'Michele']
['ASAMI', 'Seiya']
['JORGENSEN', 'Patrick']
['C

In [11]:
conn = get_db_connection()
curs = conn.cursor()

In [21]:
res = curs.execute("SELECT f.first_name, f.last_name, b.win_score, b.lose_score FROM fencers f, losers l, winners w, bouts b WHERE f.id = l.fencer_id AND l.bout_id = w.bout_id AND b.id = w.bout_id AND w.fencer_id = ?", (1,)).fetchall()

In [22]:
for r in res:
    print(f"{r['win_score']} to {r['lose_score']} against {r['first_name']} {r['last_name']}")

5 to 1 against RYU MATSUMOTO
5 to 3 against AHMED SELIM
5 to 2 against SAIF AHMED
5 to 4 against VITALIY SVYSTIL
15 to 7 against KENJI SAITO
15 to 6 against SAMUEL UNTERHAUSER
15 to 7 against KENJI SAITO
15 to 11 against MOHAMED ELSAYED
15 to 13 against RADOSLAW ZAWROTNIAK
5 to 1 against HADRIEN FAVRE
5 to 2 against DIEGO BALMORI
5 to 4 against VAINO PAAVOLAINEN
5 to 3 against AYMERICK GALLY
5 to 4 against ADRIAN DABIJA
5 to 4 against WILLIAM BISHOP
3 to 2 against MATTHEW VALKENBURG
4 to 2 against DANIEL DE MOLA
5 to 4 against ELIA DAGANI
5 to 2 against SANTIAGO JURADO SOSSA
3 to 2 against SATORU UYAMA
4 to 3 against LAURIE SHONG
5 to 2 against EDUARDO BRITO
5 to 3 against RUSLAN TYMOSHENKO
5 to 4 against ILIM BAKYTBEK UULU
5 to 2 against MIKAYEL GHAZARYAN
5 to 2 against IKUO FURUTA
15 to 9 against CHARLES-ERIC OSWALD
15 to 9 against CHARLES-ERIC OSWALD
15 to 8 against ANGEL FABREGAT
5 to 3 against JOHN EDISON RODRIGUEZ
4 to 2 against JAKOB STYMNE
5 to 1 against ALEXANDER BIRO
3 to 2 a